# Practica 3
## Maestria en Computo Aplicado

*Topicos de la Industria I*

**Nombre:** Efren Del Real Frias  
**e-mail:** efren.delreal8824@alumnos.udg.mx  
**Fecha:** Abril 15 del 2022

# Software Requirements

* Funciones que generen trayectorias tipo Brownian Motion (BM), Correlated Random Walk (CRW) y Lefy Flight (LF).
Cada una de las funcioones deberá tomar como parámetros el numero de pasos, la velocidad y posición inicial. Además la funciones para CRW  y LF deberán tomar como parámetro el coeficiente para la distribución Cauchy. Por último, la función para LF también deberá aceptar como parámetro el exponente Levy (alpha).
La función retornará un DataFrame de Pandas con todas los puntos que forman la trayectoria.

* El dashboard deberá contener al menos los siguientes elementos (Para generar el dashboard, el estudiante podrá hacer uso de la API de su elección):
* Un panel para desplegar la trayectoria en 3D
* Un panel para desplegar la métrica de elección de la trayectoria.
* Radio Buttons (u otro tipo de selector) que permita elegir el tipo de trayectoria a análizar.
* Widgets que permitan introducir los valores enteros o de punto flotante para ajustar los parámetros de las trayectorias.
Drop down menu (u otro tipo de selector) que permita elegir la métrica a calcualr de la trayectoria bajo análisis.

# Python Header

## Modules

In [1]:
import math
import pandas as pd
import numpy as np

from scipy.stats import wrapcauchy
from scipy.stats import levy_stable
from scipy.spatial import distance

import plotly.graph_objects as go

import panel as pn
pn.extension('plotly')

## Classes

In [2]:
################# http://www.pygame.org/wiki/2DVectorClass ##################
class Vec2d(object):
    """2d vector class, supports vector and scalar operators,
       and also provides a bunch of high level functions
       """
    __slots__ = ['x', 'y']

    def __init__(self, x_or_pair, y = None):
        if y == None:            
            self.x = x_or_pair[0]
            self.y = x_or_pair[1]
        else:
            self.x = x_or_pair
            self.y = y
            
    # Addition
    def __add__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x + other.x, self.y + other.y)
        elif hasattr(other, "__getitem__"):
            return Vec2d(self.x + other[0], self.y + other[1])
        else:
            return Vec2d(self.x + other, self.y + other)

    # Subtraction
    def __sub__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x - other.x, self.y - other.y)
        elif (hasattr(other, "__getitem__")):
            return Vec2d(self.x - other[0], self.y - other[1])
        else:
            return Vec2d(self.x - other, self.y - other)
    
    # Vector length
    def get_length(self):
        return math.sqrt(self.x**2 + self.y**2)
    
    # rotate vector
    def rotated(self, angle):        
        cos = math.cos(angle)
        sin = math.sin(angle)
        x = self.x*cos - self.y*sin
        y = self.x*sin + self.y*cos
        return Vec2d(x, y)

## Functions

In [3]:
import time
import functools

In [4]:
time.perf_counter()

91.454724816

In [5]:
globalDelta: float  = 0.0
def measureTimeExe(myFunc):

  @functools.wraps(myFunc)
  def wrapper(*args, **kwargs):
    global globalDelta

    tStart  = time.perf_counter()
    retObj  = myFunc(*args, **kwargs)
    tEnd    = time.perf_counter()

    tDelta = tEnd - tStart

    globalDelta += tDelta

    print(f"Finished {myFunc.__name__!r} in {tDelta:.4f} secs")
    return retObj

  return wrapper

In [6]:
def exeNTimes(myFunc, numTimes, *args, **kwargs):
  global globalDelta

  globalDelta = 0.0
  for _ in range(numTimes):
     retObj = myFunc(*args, **kwargs)


  print(f'Average: {globalDelta/ numTimes}')

  return retObj

    

  

## 1.-  Brownian Motion

In [14]:
@measureTimeExe
def trajBM_2D(n_steps=1000, speed=6, s_pos=[0, 0]):
  """
  """
  #
  vPos    = Vec2d(speed, 0.0)
  aTraj2D = np.empty(shape=(n_steps, 2))

  # Init posicion
  aTraj2D[0] = s_pos
  

  for i in range(1, n_steps):
    theta = np.random.uniform(low=-np.pi, high=np.pi)

    vPos = vPos.rotated(theta)

    aTraj2D[i][0] = aTraj2D[i - 1][0] + vPos.x
    aTraj2D[i][1] = aTraj2D[i - 1][1] + vPos.y

  
  return aTraj2D

In [15]:
 aBM2DTest = exeNTimes(trajBM_2D, 10)
 aBM2DTest.shape

Finished 'trajBM_2D' in 0.0088 secs
Finished 'trajBM_2D' in 0.0066 secs
Finished 'trajBM_2D' in 0.0068 secs
Finished 'trajBM_2D' in 0.0050 secs
Finished 'trajBM_2D' in 0.0058 secs
Finished 'trajBM_2D' in 0.0061 secs
Finished 'trajBM_2D' in 0.0049 secs
Finished 'trajBM_2D' in 0.0055 secs
Finished 'trajBM_2D' in 0.0061 secs
Finished 'trajBM_2D' in 0.0050 secs
Average: 0.006058670499999153


(1000, 2)

In [16]:
figTraj = go.Figure()

figTraj.add_trace(
    go.Scatter3d(
        x=aBM2DTest[:,0],
        y=aBM2DTest[:,1],
        z=np.arange(len(aBM2DTest)),
        marker=dict(size=2),
        line=dict(color='blue', width=2),
        mode='lines',
        name=f'steps = {len(aBM2DTest)}',
        showlegend=True
      )
)

figTraj.show()

## 2.- Correlated Random Walk

In [18]:
@measureTimeExe
def trajCRW_2D(n_steps=1000, speed=6, s_pos=[0, 0], exp_c = 0.5):
  """
  """
  #
  vPos    = Vec2d(speed, 0.0)
  aTraj2D = np.empty(shape=(n_steps, 2))

  # Init posicion
  aTraj2D[0] = s_pos
  

  for i in range(1, n_steps):

    #
    theta = wrapcauchy.rvs(exp_c)

    # 
    vPos = vPos.rotated(theta)

    aTraj2D[i][0] = aTraj2D[i - 1][0] + vPos.x
    aTraj2D[i][1] = aTraj2D[i - 1][1] + vPos.y

  
  return aTraj2D

In [19]:
 aCRW2DTest = exeNTimes(trajCRW_2D, 10)
 aCRW2DTest.shape

Finished 'trajCRW_2D' in 0.1072 secs
Finished 'trajCRW_2D' in 0.1029 secs
Finished 'trajCRW_2D' in 0.1003 secs
Finished 'trajCRW_2D' in 0.0991 secs
Finished 'trajCRW_2D' in 0.1090 secs
Finished 'trajCRW_2D' in 0.0984 secs
Finished 'trajCRW_2D' in 0.1056 secs
Finished 'trajCRW_2D' in 0.1036 secs
Finished 'trajCRW_2D' in 0.1050 secs
Finished 'trajCRW_2D' in 0.0741 secs
Average: 0.10050253260003501


(1000, 2)

In [20]:
figTraj = go.Figure()

figTraj.add_trace(
    go.Scatter3d(
        x=aCRW2DTest[:,0],
        y=aCRW2DTest[:,1],
        z=np.arange(len(aCRW2DTest)),
        marker=dict(size=2),
        line=dict(color='blue', width=2),
        mode='lines',
        name=f'steps = {len(aCRW2DTest)}',
        showlegend=True
      )
)

figTraj.show()

## 3.- Levy Flight

In [154]:
@measureTimeExe
def trajLF_2D(n_steps=1000, speed=6, s_pos=[0, 0], exp_alpha = 0.7, exp_beta = 0.0, std_motion=3):
  """
  """
  
  #
  vPos    = Vec2d(speed, 0.0)
  aTraj2D = np.empty(shape=(n_steps, 2))

  # Init posicion
  aTraj2D[0] = s_pos
  

  for i in range(1, n_steps):

    theta:    float = 0.0
    step_size:  int = 0
    
    while not (bool(theta) and bool(step_size)):
      theta = np.random.uniform(-np.pi, np.pi)
      step_size = levy_stable.rvs(alpha=exp_alpha, beta=exp_beta, loc=std_motion)
      step_size = int(np.floor(step_size))

    # 
    vPos = vPos.rotated(theta)

    aTraj2D[i][0] = aTraj2D[i - 1][0] + ( vPos.x * step_size )
    aTraj2D[i][1] = aTraj2D[i - 1][1] + ( vPos.y * step_size )

  
  return aTraj2D

In [155]:
 aLF2DTest = exeNTimes(trajLF_2D, 10)
 aLF2DTest.shape

Finished 'trajLF_2D' in 0.6501 secs
Finished 'trajLF_2D' in 0.5993 secs
Finished 'trajLF_2D' in 0.5955 secs
Finished 'trajLF_2D' in 0.5969 secs
Finished 'trajLF_2D' in 0.6218 secs
Finished 'trajLF_2D' in 0.6544 secs
Finished 'trajLF_2D' in 1.2601 secs
Finished 'trajLF_2D' in 0.6138 secs
Finished 'trajLF_2D' in 0.6162 secs
Finished 'trajLF_2D' in 0.6242 secs
Average: 0.683242620299825


(1000, 2)

In [156]:
figTraj = go.Figure()

figTraj.add_trace(
    go.Scatter3d(
        x=aLF2DTest[:,0],
        y=aLF2DTest[:,1],
        z=np.arange(len(aLF2DTest)),
        marker=dict(size=2),
        line=dict(color='blue', width=2),
        mode='lines',
        name=f'steps = {len(aLF2DTest)}',
        showlegend=True
      )
)

figTraj.show()

##  4.- Compute Path Lenght
$$
Euclidean(A, B) = \sqrt{(x_{2} - x_{1})^2  + (y_{2} - y_{1})^2}
$$

$where:$  


*   $A(x_{1}, y_{1})$
*   $B(x_{2}, y_{2})$




In [211]:
###############################################################################################
# Compute Path Lenght
###############################################################################################
@measureTimeExe
def compute_path_lenght(aRW2D):
  """
  Arguments:
        aRW2D:    np.array  - Random Walk 2D size(rows = n, columns = 2)

  Returns:
        pl:       np.array  - Path Lenght numpy array size(rows = n, columns = 1)

  See:
        https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.distance.euclidean.html
  """
  disEuclidean = np.array([ distance.euclidean( aRW2D[i - 1], aRW2D[i] )
                            for i in range(1, aRW2D.shape[0]) ])
  aPathLen = np.cumsum(disEuclidean)

  return aPathLen

In [209]:
try:
  plBM = exeNTimes(compute_path_lenght, 10, aBM2DTest)
  print(plBM.shape)
except NameError:
  print('DataFrame is no defined yet')

Finished 'compute_path_lenght' in 0.0178 secs
Finished 'compute_path_lenght' in 0.0103 secs
Finished 'compute_path_lenght' in 0.0104 secs
Finished 'compute_path_lenght' in 0.0106 secs
Finished 'compute_path_lenght' in 0.0115 secs
Finished 'compute_path_lenght' in 0.0101 secs
Finished 'compute_path_lenght' in 0.0102 secs
Finished 'compute_path_lenght' in 0.0105 secs
Finished 'compute_path_lenght' in 0.0117 secs
Finished 'compute_path_lenght' in 0.0107 secs
Average: 0.011402546699810046
(999,)


In [208]:
try:
  plCRW = exeNTimes(compute_path_lenght, 10, aCRW2DTest)
  print(plCRW.shape)
except NameError:
  print('DataFrame is no defined yet')

Finished 'compute_path_lenght' in 0.0118 secs
Finished 'compute_path_lenght' in 0.0102 secs
Finished 'compute_path_lenght' in 0.0101 secs
Finished 'compute_path_lenght' in 0.0101 secs
Finished 'compute_path_lenght' in 0.0171 secs
Finished 'compute_path_lenght' in 0.0142 secs
Finished 'compute_path_lenght' in 0.0116 secs
Finished 'compute_path_lenght' in 0.0113 secs
Finished 'compute_path_lenght' in 0.0103 secs
Finished 'compute_path_lenght' in 0.0110 secs
Average: 0.011775792499975068
(999,)


In [207]:
try:
  plLevy = exeNTimes(compute_path_lenght, 10, aLF2DTest)
  print(plLevy.shape)
except NameError:
  print('DataFrame is no defined yet')

Finished 'compute_path_lenght' in 0.0190 secs
Finished 'compute_path_lenght' in 0.0173 secs
Finished 'compute_path_lenght' in 0.0175 secs
Finished 'compute_path_lenght' in 0.0225 secs
Finished 'compute_path_lenght' in 0.0169 secs
Finished 'compute_path_lenght' in 0.0167 secs
Finished 'compute_path_lenght' in 0.0202 secs
Finished 'compute_path_lenght' in 0.0182 secs
Finished 'compute_path_lenght' in 0.0178 secs
Finished 'compute_path_lenght' in 0.0183 secs
Average: 0.01844368290012426
(999,)


In [210]:
# Init figure
fig_path_length = go.Figure()

timePlot = np.arange(len(plBM))
# first trace MSD_BM
## start - Add your code here
fig_path_length.add_trace(
    go.Scatter(
        x=timePlot,
        y=plBM[:],
        marker=dict(size=2),
        line = dict(width=2),
        mode = 'lines',
        name = 'BM',
        showlegend = True
))       
## end - Add your code here


# Second trace MSD_CRW
## start - Add your code here
fig_path_length.add_trace(
    go.Scatter(
        x=timePlot,
        y=plCRW[:],
        marker=dict(size=2),
        line = dict(width=2),
        mode = 'lines',
        name = 'CRW',
        showlegend = True
))         
## end - Add your code here

# Second trace MSD_CRW
## start - Add your code here
fig_path_length.add_trace(
    go.Scatter(
        x=timePlot,
        y=plLevy[:],
        marker=dict(size=2),
        line = dict(width=2),
        mode = 'lines',
        name = 'Levy',
        showlegend = True
))         
## end - Add your code here

fig_path_length.show()

## 5.- Mean Squared Displacement

$$
MSD = \frac{1}{N - n} \sum_{i=1}^{N-n}(r_{i+n} -r_{i})^2
$$

where $n = 1,...., N - 1$

In [213]:
###############################################################################################
# mean_squared_displacement
###############################################################################################
def mean_squared_displacement(aRW2D, tau: int) -> float:
  """
  Arguments:
        aRW2D:  np.array    - Random Walk 2D size(rows = n, columns = 2)
        tau:    integer     - Iterable number to displace two points (called "n" in above equation)
  Returns:
        msd:    float       - mean squared displacement value
  """
  aDisplacement = np.array([ distance.euclidean( aRW2D[i - tau], aRW2D[i] ) 
                            for i in range(tau, aRW2D.shape[0]) ])
  
  msd: float = np.mean(np.power(aDisplacement, 2))
  
  return msd

In [214]:
@measureTimeExe
def compute_MSD(aRW2D):
  """
  Arguments:
        aRW2D:  np.array    - Random Walk 2D size(rows = n, columns = 2)
  Returns:
        aMSD:   np.array    - Array MSD's metrics
  """
  aMSD = np.empty(shape=(0))

  for tau in range(1, aRW2D.shape[0]):
      aMSD = np.concatenate( (aMSD, [mean_squared_displacement(aRW2D, tau)]), axis=0)
      
  return aMSD

In [215]:
try:
  msdBM = exeNTimes(compute_MSD, 10, aBM2DTest)
except NameError:
  print('DataFrame is no defined yet')

Finished 'compute_MSD' in 5.1557 secs
Finished 'compute_MSD' in 5.1398 secs
Finished 'compute_MSD' in 5.1167 secs
Finished 'compute_MSD' in 5.1976 secs
Finished 'compute_MSD' in 7.9911 secs
Finished 'compute_MSD' in 6.6627 secs
Finished 'compute_MSD' in 7.4168 secs
Finished 'compute_MSD' in 5.9993 secs
Finished 'compute_MSD' in 6.1502 secs
Finished 'compute_MSD' in 5.6455 secs
Average: 6.047541066199938


In [216]:
try:
  msdCRW = exeNTimes(compute_MSD, 10, aCRW2DTest)
except NameError:
  print('DataFrame is no defined yet')

Finished 'compute_MSD' in 5.0109 secs
Finished 'compute_MSD' in 4.9086 secs
Finished 'compute_MSD' in 5.4779 secs
Finished 'compute_MSD' in 5.9456 secs
Finished 'compute_MSD' in 5.0597 secs
Finished 'compute_MSD' in 5.0258 secs
Finished 'compute_MSD' in 5.0891 secs
Finished 'compute_MSD' in 5.5472 secs
Finished 'compute_MSD' in 6.1460 secs
Finished 'compute_MSD' in 5.1248 secs
Average: 5.333563870299895


In [217]:
try:
  msdLF = exeNTimes(compute_MSD, 10, aLF2DTest)
except NameError:
  print('DataFrame is no defined yet')

Finished 'compute_MSD' in 5.0937 secs
Finished 'compute_MSD' in 6.1271 secs
Finished 'compute_MSD' in 6.1748 secs
Finished 'compute_MSD' in 10.0079 secs
Finished 'compute_MSD' in 6.0883 secs
Finished 'compute_MSD' in 5.9260 secs
Finished 'compute_MSD' in 4.9816 secs
Finished 'compute_MSD' in 5.0232 secs
Finished 'compute_MSD' in 5.0570 secs
Finished 'compute_MSD' in 5.1293 secs
Average: 5.960887438000009


In [218]:
# Init figure
fig_path_length = go.Figure()

timePlot = np.arange(len(msdCRW))
# first trace MSD_BM
## start - Add your code here
fig_path_length.add_trace(
    go.Scatter(
        x=timePlot,
        y=msdBM[:],
        marker=dict(size=2),
        line = dict(width=2),
        mode = 'lines',
        name = 'MSD BM',
        showlegend = True
))       
## end - Add your code here


# Second trace MSD_CRW
## start - Add your code here
fig_path_length.add_trace(
    go.Scatter(
        x=timePlot,
        y=msdCRW[:],
        marker=dict(size=2),
        line = dict(width=2),
        mode = 'lines',
        name = 'MSD CRW',
        showlegend = True
))         
## end - Add your code here

# Second trace MSD_CRW
## start - Add your code here
fig_path_length.add_trace(
    go.Scatter(
        x=timePlot,
        y=msdLF[:],
        marker=dict(size=2),
        line = dict(width=2),
        mode = 'lines',
        name = 'MSD LF',
        showlegend = True
))         
## end - Add your code here

fig_path_length.show()

## Turning angle

In [ ]:
###############################################################################################
# Turning angle
###############################################################################################
def turning_angle(vec_a, vec_b, vec_c):
    """
    Arguments:
        vec_a: First detection coordinates
        vec_b: Second detection coordinates
        vec_c: Third detection coordinates
    Returns:
        theta: Turning angle 
    """
    ab = vec_b-vec_a
    norm_ab = np.linalg.norm(ab)
    
    bc = vec_c-vec_b
    norm_bc = np.linalg.norm(bc)

    dot_p = np.dot(ab, bc)
    
    cross_p = np.cross(ab, bc)
    orient = np.sign(cross_p)
    if orient == 0:
        orient = 1
    
    cos_theta = dot_p/(norm_ab*norm_bc+np.finfo(float).eps)
    theta = np.arccos(np.around(cos_theta,4)) * orient
    return theta

In [ ]:
###############################################################################################
# Compute Path Lenght
###############################################################################################
def levy_compute_step_lenght(levy_df):
  """
  Arguments:
        levy_df:                    pd.DataFrame - 
        Levy Walk pandas DataFrame size(rows = n, columns = 2)

  Returns:
        (aux_ta_Levy, aux_sl_Levy):  Tuple(np.array, np.array) - 
        Levy angle and Levy step lengh 
  """
  # Define Local Variables
  theta:        float = 0.0        
  prev_theta:   float = (3 * np.pi)  #
  prev_index:     int = 0
  cal_vel:       bool = True         # Calculate Velocity
  velocity:     float = 0.0

  # aux to store turning angles
  aux_ta_Levy = np.empty(shape=(0))
  # aux to store step-lengths
  aux_sl_Levy = np.empty(shape=(0))

  # Start to calculate over all DataFrame row values
  for index, _ in levy_df[1:-1].iterrows():

      a = levy_df.iloc[index - 1]
      b = levy_df.iloc[index]
      c = levy_df.iloc[index + 1]

      theta = turning_angle(a, b, c)
      
      # Ignore 0.0 since is still in the same direction
      if (theta != 0.0):

        if (prev_theta !=  theta):
          
          # Add to the end the new direction
          aux_ta_Levy = np.concatenate( (aux_ta_Levy, [theta]), axis=0)

          # Gets path lenght - This value includes velocity
          pl = distance.euclidean( levy_df.iloc[prev_index] , levy_df.iloc[index] )

          # Add to the end new path lenght
          pl = np.round(pl)
          aux_sl_Levy = np.concatenate ( (aux_sl_Levy, [pl]) , axis=0)


          # Keeps current index and angle to the next step
          prev_index = index
          prev_theta = theta
      else:
        # theta == 0.0
        if cal_vel:
          # Gets velocity from two close steps
          velocity = distance.euclidean( levy_df.iloc[prev_index] , levy_df.iloc[index] )
          
          # Clean in order to only execute one time
          cal_vel = False

  
  if not cal_vel:
    # We need to divite over the velocity in order
    # to get Levy step lenght
    aux_sl_Levy = aux_sl_Levy / velocity

  return aux_ta_Levy, aux_sl_Levy

## Widgets

In [219]:
wgTrajType = pn.widgets.RadioButtonGroup(
    name='Trajectory type', options=['BM', 'CRW', 'LF'], 
    button_type='default' # [default, primary, success, warning, danger]
    )


wgTrajType

RadioButtonGroup(name='Trajectory type', options=['BM', 'CRW', 'LF'], value='BM')

In [220]:
wgMtcrType = pn.widgets.Select(value='PL', options=['PL','MSD'], name='Metrics type')
wgMtcrType

Select(name='Metrics type', options=['PL', 'MSD'], value='PL')

In [221]:
wgNsteps = pn.widgets.IntSlider(name='number of steps', start=50, end=1000, step=50, value=200)

wgNsteps

IntSlider(end=1000, name='number of steps', start=50, step=50, value=200)

In [222]:
wgSpeed = pn.widgets.IntSlider(name='Speed', start=1, end=10, step=1, value=4)

wgSpeed

IntSlider(end=10, name='Speed', start=1, value=4)

In [223]:
wgXInitPos = pn.widgets.IntInput(name='xInitPos', value=1, step=1, start=0, end=100)
wgXInitPos

IntInput(end=100, name='xInitPos', start=0, value=1)

In [224]:
wgYInitPos = pn.widgets.IntInput(name='yInitPos', value=0, step=1, start=0, end=100)
wgYInitPos

IntInput(end=100, name='yInitPos', start=0)

In [225]:
@pn.depends(wgXInitPos, wgYInitPos)
def display_InitPos(xVal, yVal):
  return (xVal, yVal)

In [226]:
pn.Row(wgXInitPos, wgYInitPos, display_InitPos)

Row
    [0] IntInput(end=100, name='xInitPos', start=0, value=1)
    [1] IntInput(end=100, name='yInitPos', start=0)
    [2] ParamFunction(function)

# Implementation

In [228]:
# Init eGererate Random Walk Array
aGenRW = np.empty(shape=(0))


@pn.depends(wgTrajType, wgNsteps, wgSpeed, wgXInitPos, wgYInitPos)
def main_plotTraj(
    trajType: str,
    Nsteps:   int, 
    speedVal: int, 
    xPos:     int, 
    yPos:     int):
  """
  """

  #
  global aGenRW

  #
  figTraj = go.Figure()

  #
  if ('BM' == trajType):
    aGenRW = trajBM_2D(n_steps=Nsteps, speed=speedVal , s_pos=[xPos, yPos])
  elif ('CRW' == trajType):
    aGenRW = trajCRW_2D(n_steps=Nsteps, speed=speedVal , s_pos=[xPos, yPos])
  elif ('LF' == trajType):
    aGenRW = trajLF_2D(n_steps=Nsteps, speed=speedVal , s_pos=[xPos, yPos])


  # Plot trajectory in 3D space
  figTraj.add_trace(
      go.Scatter3d(
          x = aGenRW[:,0],
          y = aGenRW[:,1],
          z = np.arange(len(aGenRW)),
          marker=dict(size=2),
          line=dict(color='blue', width=2),
          mode='lines',
          name=f'steps = {Nsteps}',
          showlegend=True
      )
  )

  return figTraj

In [229]:
@pn.depends(wgMtcrType)
def main_plotMtcr(mtcrType: str):

  #
  global aGenRW

  #
  metric = np.empty(shape=(0))
  fig_mctr = go.Figure()



  #  
  if ('PL' == mtcrType):
    metric  = compute_path_lenght(aGenRW)
  elif ('MSD' == mtcrType):
    metric  = compute_MSD(aGenRW)



  # Plot trajectory
  fig_mctr.add_trace(
      go.Scatter(
          x=np.arange(len(metric)),
          y=metric,
          marker=dict(size=2),
          line = dict(width=2),
          mode = 'lines',
          name = f'{mtcrType}',
          showlegend = True
  ))

  return fig_mctr

In [230]:
# Dashboard
wgPanel = pn.Row(
    pn.Column(
        'A',
        wgTrajType, 
        wgNsteps,
        pn.Row(wgXInitPos, wgYInitPos),
        wgSpeed),
    pn.Column(
        '3D Trajectory',
        main_plotTraj),
    pn.Column(
        'Metrics',
        wgMtcrType,
        main_plotMtcr)
)
wgPanel

Finished 'trajBM_2D' in 0.0016 secs
Finished 'compute_path_lenght' in 0.0022 secs


Row
    [0] Column
        [0] Markdown(str)
        [1] RadioButtonGroup(name='Trajectory type', options=['BM', 'CRW', 'LF'], value='BM')
        [2] IntSlider(end=1000, name='number of steps', start=50, step=50, value=200)
        [3] Row
            [0] IntInput(end=100, name='xInitPos', start=0, value=4)
            [1] IntInput(end=100, name='yInitPos', start=0, value=14)
        [4] IntSlider(end=10, name='Speed', start=1, value=4)
    [1] Column
        [0] Markdown(str)
        [1] ParamFunction(function)
    [2] Column
        [0] Markdown(str)
        [1] Select(name='Metrics type', options=['PL', 'MSD'], value='PL')
        [2] ParamFunction(function)

Finished 'trajLF_2D' in 0.1176 secs 
Finished 'compute_MSD' in 0.2171 secs 
Finished 'compute_path_lenght' in 0.0041 secs 
Finished 'compute_MSD' in 0.2195 secs 
Finished 'compute_path_lenght' in 0.0022 secs